In [1]:
import json, requests
import pandas as pd
from bs4 import BeautifulSoup

base_url = r'/home/amit/DataScienceProject/PredictDroughts/'

In [2]:
def returnLastWeekDate():
    df = pd.read_csv(base_url + 'files/drought.csv')
    weekLines = len(df.loc[df['State'] == 'Alabama'])
    return df['Week'][weekLines - 1]

lastWeekDate = returnLastWeekDate()
lastWeekDate = lastWeekDate.replace('-', '')
lastWeekDate

'20210615'

In [3]:
def getLonAndLat():
    url = r'https://developers.google.com/public-data/docs/canonical/states_csv'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    div = soup.find('table')
    table = pd.read_html(str(div))[0]
    table.sort_values('name', ascending=True, inplace=True)
    lon = table['longitude'].to_list()
    lat = table['latitude'].to_list()
    return lon, lat

lon, lat = getLonAndLat()
locations = list(zip(lat, lon))

In [4]:
URL = r"https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?request=execute&identifier=SinglePoint&tempAverage=DAILY&parameters=WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,T2M_MAX,WS50M,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET,PRECTOT&startDate=20000104&endDate={lastWeekDate}&lat={latitude}&lon={longitude}&outputList={output}&userCommunity=SSE"

dfList = []

for latitude, longitude in locations:
    TIME = []
    LON = []
    LAT = []
    WS10M_MIN = []
    QV2M = []
    T2M_RANGE = []
    WS10M = []
    T2M = []
    WS50M_MIN = []
    T2M_MAX = []
    WS50M = []
    TS = []
    WS50M_RANGE = []
    WS50M_MAX = []
    WS10M_MAX = []
    WS10M_RANGE = []
    PS = []
    T2MDEW = []
    T2M_MIN = []
    T2MWET = []
    PRECTOT = []

    api_request_url = URL.format(lastWeekDate=lastWeekDate, longitude=longitude, latitude=latitude, output="JSON")
    json_response = json.loads(requests.get(api_request_url).content)
    params = json_response['features'][0]['properties']['parameter']

    for index, date in enumerate(params['PRECTOT'].keys()):
        date = str(date)
        if index % 7 == 0:
            TIME.append(date)
            LON.append(longitude)
            LAT.append(latitude)
            PRECTOT.append(params['PRECTOT'][date])
            WS10M_MIN.append(params['WS10M_MIN'][date])
            QV2M.append(params['QV2M'][date])
            T2M_RANGE.append(params['T2M_RANGE'][date])
            WS10M.append(params['WS10M'][date])
            T2M.append(params['T2M'][date])
            WS50M_MIN.append(params['WS50M_MIN'][date])
            T2M_MAX.append(params['T2M_MAX'][date])
            WS50M.append(params['WS50M'][date])
            TS.append(params['TS'][date])
            WS50M_RANGE.append(params['WS50M_RANGE'][date])
            WS50M_MAX.append(params['WS50M_MAX'][date])
            WS10M_MAX.append(params['WS10M_MAX'][date])
            WS10M_RANGE.append(params['WS10M_RANGE'][date])
            PS.append(params['PS'][date])
            T2MDEW.append(params['T2MDEW'][date])
            T2M_MIN.append(params['T2M_MIN'][date])
            T2MWET.append(params['T2MWET'][date])
            
        if date == lastWeekDate:
            index = 0

    data = {
        'week': TIME,
        'Longitude': LON,
        'Latitude': LAT,
        'PRECTOT': PRECTOT,
        'WS10M_MIN': WS10M_MIN,
        'QV2M': QV2M,
        'T2M_RANGE': T2M_RANGE,
        'WS10M': WS10M,
        'T2M': T2M,
        'WS50M_MIN': WS50M_MIN,
        'T2M_MAX': T2M_MAX,
        'WS50M': WS50M,
        'TS': TS,
        'WS50M_RANGE': WS50M_RANGE,
        'WS50M_MAX': WS50M_MAX,
        'WS10M_MAX': WS10M_MAX,
        'WS10M_RANGE': WS10M_RANGE,
        'PS': PS,
        'T2MDEW': T2MDEW,
        'T2M_MIN': T2M_MIN,
        'T2MWET': T2MWET
    }

    df = pd.DataFrame(data)
    dfList.append(df)

In [5]:
dataframe = pd.DataFrame()
for df in dfList:
    dataframe = dataframe.append(df)
dataframe.reset_index(drop=True, inplace=True)
dataframe

,week,Longitude,Latitude,PRECTOT,WS10M_MIN,QV2M,T2M_RANGE,WS10M,T2M,WS50M_MIN,...,WS50M,TS,WS50M_RANGE,WS50M_MAX,WS10M_MAX,WS10M_RANGE,PS,T2MDEW,T2M_MIN,T2MWET
0,20000104,-86.902298,32.318231,12.35,2.14,6.84,15.39,3.80,12.27,3.55,...,6.75,12.18,5.24,8.80,5.44,3.30,100.72,6.95,2.85,6.97
1,20000111,-86.902298,32.318231,0.97,1.18,7.09,13.70,1.92,11.76,2.08,...,4.03,11.15,4.09,6.18,2.51,1.33,100.83,8.89,5.91,8.89
2,20000118,-86.902298,32.318231,1.80,1.59,9.91,6.50,2.54,15.00,3.54,...,5.14,14.94,2.75,6.29,3.87,2.28,100.82,13.94,11.96,13.94
3,20000125,-86.902298,32.318231,0.00,2.20,2.15,10.89,3.18,0.11,3.89,...,5.58,0.24,3.71,7.61,4.35,2.16,100.53,-7.34,-4.39,-7.14
4,20000201,-86.902298,32.318231,0.01,0.73,3.38,15.38,1.89,2.35,0.92,...,3.91,2.15,5.12,6.04,2.41,1.69,101.43,-1.58,-3.99,-1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58235,20210518,-107.290284,43.075968,0.00,0.47,6.54,17.38,2.39,16.42,0.72,...,3.57,17.56,6.01,6.73,3.36,2.89,80.75,4.49,6.61,10.45
58236,20210525,-107.290284,43.075968,0.11,0.95,5.07,16.60,4.37,11.96,1.69,...,5.89,13.43,7.49,9.18,6.45,5.50,81.35,1.03,3.02,6.50
58237,20210601,-107.290284,43.075968,1.19,1.75,7.33,12.02,3.18,16.19,2.93,...,4.55,17.39,3.61,6.54,4.33,2.58,81.70,6.35,9.74,11.27
58238,20210608,-107.290284,43.075968,0.00,0.49,6.30,17.62,4.97,20.90,1.54,...,6.99,21.32,13.17,14.71,11.86,11.36,80.92,3.91,12.01,12.41


In [6]:
dataframe.to_csv(base_url + 'files/meteorological.csv')